# STAT 440 Statistical Data Management - Fall 2021
## Week 12 Notes
### Created by Christopher Kinson


***


## Table of Contents

- [More SQL](#more)
- [Summarizing data](#summarizing)
- [Combining data](#combining)  
- [Validating data](#validating)
  - [Strategy 1 - Filtering and arranging](#stra1)
  - [Strategy 2 - Counting frequencies and duplicates](#stra2)
  - [Strategy 3 - Computing summary statistics](#stra3)
- [Cleaning data](#cleaning)
  - [Approach 1 - Removing duplicate observations](#appr1)
  - [Approach 2 - Fixing rounding errors and inconsistent units of measurement](#appr2)
  - [Approach 3 - Removing or replacing missing values](#appr3)
  - [Approach 4 - Limiting a distribution to its realistic set of observations](#appr4)
  - [Approach 5 - Correcting and subsetting with dates](#appr5)
  - [Approach 6 - Correcting misspelled words, abbreviations, or text cases](#appr6)
- [SQL Subqueries](#subqueries)
  - [Example 1](#example01)
  - [Example 2](#example02)
  - [Example 3](#example03)
  - [Example 4](#example04)


***


## <a name="more"></a>More SQL

We continue exploring the many actions we can take in SQL and its query system.

## <a name="summarizing"></a>Summarizing data

Another important aspect of data wrangling is to summarize or aggregate data. This may also be considered as applying summary functions to grouped data aka "group processing". Grouped data can be any data with a categorical variable or factor as a column. This task comes in handy when we want to know statistical or numeric values for each member of a group. To accomplish summarization, sometimes we can leverage the way the data are arranged (or sorted). Other times, the arrangement has no bearing on our ability to aggregate.

Working with the City of Urbana's [Rental Inspection Grades Listing Data as tab-separated .txt - GHE](https://github-dev.cs.illinois.edu/stat440-fa21/stat440-fa21-course-content/raw/master/data/rental-inspections-grades-data03.txt) or [Rental Inspection Grades Listing Data as tab-separated .txt - Box](https://uofi.box.com/shared/static/0j11not1cqmonbrwy0l8zmzr27fafm2j.txt), we use SQL's `GROUP BY` and `HAVING` clauses for group processing. Let's compute the proportion for each inspection grade.


In [5]:
import pandas as pd
RentalsData = pd.read_csv('https://uofi.box.com/shared/static/0j11not1cqmonbrwy0l8zmzr27fafm2j.txt',sep='\t')

In [6]:
import sqlite3
conn = sqlite3.connect("stat440_fa21_notes_week12_1.db") 
conn.commit()
RentalsData.to_sql('RentalsData', conn, if_exists='replace', index = False)

C:\Users\kinson2\Anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [7]:
pd.read_sql_query("SELECT Grade, COUNT(Grade) AS count_grade, COUNT(Grade)/1730.0 AS grade_proportion \
       FROM RentalsData \
       GROUP BY Grade;", conn)

,Grade,count_grade,grade_proportion
0,Class A,155,0.089595
1,Class B,1442,0.833526
2,Class C,127,0.073410
3,Class D,3,0.001734
4,Class F,2,0.001156
5,Class N,1,0.000578


***


## <a name="combining"></a>Combining data

Merging (or joining) usually implies combining two or more objects with different columns of information into one single object. This merging would require each of the different data objects to have one column in common with a unique identifying information such as an ID variable or geographic location. There are at least 3 situations that can occur when merging objects. 

In SQL, binding (or appending) is the act of combining two or more objects by stacking one on top of the other using `UNION` or stacking one next to the other with SQL joins such as `FULL JOIN` or `INNER JOIN`.

1. Observations in the two (or more) separate objects could not match each other.

**Data 1**  

ID | Salary
---|---
A | \$10K
B | \$11K
D | \$12K

**Data 2**  

ID | Number
---|---
C | 2175551234
E | 2175551235
F | 2175551236

**Merged Data**  

ID | Salary | Number
---|---|---
A | \$10K |
B | \$11K |
D | \$12K |
C |  | 2175551234
E |  | 2175551235
F |  | 2175551236

2. Observations in the two (or more) separate objects could match each other one-to-one.

**Data 1**  

ID | Salary
---|---
A | \$10K
B | \$11K
D | \$12K

**Data 2**  

ID | Number
---|---
A | 2175551214
B | 2175551224
D | 2175551244

**Merged Data**  

ID | Salary | Number
---|---|---
A | \$10K | 2175551214
B | \$11K | 2175551224
D | \$12K | 2175551244

3. Observations in the two (or more) separate objects could match each other one-to-many (or many-to-one).

**Data 1**  

ID | Salary
---|---
A | \$10K
D | \$12K

**Data 2** 

ID | Number
---|---
A | 2175551214
A | 2175551204
D | 2175551244

**Merged Data**  

ID | Salary | Number
---|---|---
A | \$10K | 2175551214
A | \$10K | 2175551204
D | \$12K | 2175551244

How we merge (or join) the data depends on which of the three situations is intended for the data management. Only keeping the matches (#2 and #3 above) could be accomplished using an inner join (`INNER JOIN`) in SQL. Keeping the matches (#2 and #3 above) and non-matches (#1 above) could be accomplished using a full join (`FULL JOIN`) in SQL. ONe differentiator in SQL is that the column names do not need to be the same in order to join two or more datasets.

Let's combine owner addresses with the `RentalsData` as [owners-addresses .csv - GHE](https://github-dev.cs.illinois.edu/stat440-fa21/stat440-fa21-course-content/raw/master/data/owners-addresses.csv) or [owners-addresses .csv - Box](https://uofi.box.com/shared/static/u6coxibtzx3mith23bzk4rysu923g160.csv) with an inner join. Doing this combining is quite simple but first we need to mutate the owners addresses data with the Parcel Number column. **Notice the subtle mutated column of Parcel Number here. This is because the owners addresses data did not have the Parcel Number column to begin with. The mutating we've done in SQL has been a function of existing columns in a dataset.**

In [8]:
owners_addresses = pd.read_csv('https://uofi.box.com/shared/static/u6coxibtzx3mith23bzk4rysu923g160.csv')
owners_addresses.loc[:,'Parcel Number'] = rentals['Parcel Number']
owners_addresses.to_sql('owners_addresses', conn, if_exists='replace', index = False)
pd.read_sql_query("SELECT * \
    FROM RentalsData, owners_addresses \
    WHERE RentalsData.`Parcel Number` = owners_addresses.`Parcel Number`", conn)

C:\Users\kinson2\Anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Mappable Address,value,Parcel Number
0,607 1/2 Glover Avenue,9.221162e+11,7/24/2015,Class B,Expired,10/14/2021,"607 1 2 Glover Avenue\r\nUrbana, IL\r\n(40.108...","CORA MAE PROPERTIES LLC, \r\nLUKE SHERMAN\r\nP...",922116177018
1,1302 1/2 Hill Street,9.121074e+11,8/17/2011,Class B,Issued,10/14/2021,"1302 1 2 Hill Street\r\nUrbana, IL\r\n(40.1193...","WOMACK, DEBORAH J & MICHAEL\r\n803 N OAKWOOD S...",912107406011
2,212 1/2 Central Avenue,9.121084e+11,4/26/2010,Class B,Issued,None,"212 1 2 Central Avenue\r\nUrbana, IL","RUBIN, RACHAEL\r\n212 N CENTRAL AVE\r\nURBANA,...",912108383001
3,801 1/2 East Harding Drive,9.321212e+11,6/12/2013,Class B,Issued,10/14/2021,"801 1 2 East Harding Drive\r\nUrbana, IL\r\n(4...","HARPER, CRAIG & JAMES E\r\n1173 COUNTY ROAD 24...",932121153003
4,1003 1/2 East Harding Drive,9.321212e+11,7/8/2013,Class B,Issued,10/14/2020,"1003 1 2 East Harding Drive\r\nUrbana, IL\r\n(...","WAMPLER, JOSEPH\r\nCOLONY PROPERTY MANAGEMENT\...",932121153010
...,...,...,...,...,...,...,...,...,...
1725,3026 East Stillwater Landing Unit 101,9.321224e+11,12/18/2017,Class B,Issued,10/14/2021,"3026 East Stillwater Landing\r\nUrbana, IL\r\n...","MINCONE, SANDY K\r\n22210 TAHOE CT\r\nSANTA CL...",932122406009
1726,1108 South Busey Avenue,9.321173e+11,12/16/2019,Class B,Issued,10/14/2021,"1108 South Busey Avenue\r\nUrbana, IL\r\n(40.1...","DOYLE, KIP & SHERI\r\n906 W DANIEL ST\r\nCHAMP...",932117307003
1727,806 Harvey Street,9.121074e+11,11/4/2011,Class B,Issued,10/14/2021,"806 Harvey Street\r\nUrbana, IL\r\n(-88.2215, ...","JONES PROPERTY MANAGEMENT, \r\n2516 PINEHURST ...",912107428001
1728,1302 East Michigan Avenue,9.221164e+11,4/18/2016,Class B,Issued,None,"1302 East Michigan Avenue\r\nUrbana, IL\r\n(-8...","DILLMAN, RONALD L\r\n906 E MICHIGAN AVE\r\nURB...",922116376032


A more explicit inner join in SQL with the `INNER JOIN`, `ON`, and `AS` keywords can be seen below.

In [9]:
pd.read_sql_query("SELECT `Property Address`, rd.`Parcel Number`, `Inspection Date`, Grade, \
                  `License Status`, `Expiration Date`, `Mappable Address`, oa.value AS Owner \
                  FROM RentalsData \
                  AS rd INNER JOIN owners_addresses AS oa ON rd.`Parcel Number`=oa.`Parcel Number`",conn)

,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Mappable Address,Owner
0,607 1/2 Glover Avenue,9.221162e+11,7/24/2015,Class B,Expired,10/14/2021,"607 1 2 Glover Avenue\r\nUrbana, IL\r\n(40.108...","CORA MAE PROPERTIES LLC, \r\nLUKE SHERMAN\r\nP..."
1,1302 1/2 Hill Street,9.121074e+11,8/17/2011,Class B,Issued,10/14/2021,"1302 1 2 Hill Street\r\nUrbana, IL\r\n(40.1193...","WOMACK, DEBORAH J & MICHAEL\r\n803 N OAKWOOD S..."
2,212 1/2 Central Avenue,9.121084e+11,4/26/2010,Class B,Issued,None,"212 1 2 Central Avenue\r\nUrbana, IL","RUBIN, RACHAEL\r\n212 N CENTRAL AVE\r\nURBANA,..."
3,801 1/2 East Harding Drive,9.321212e+11,6/12/2013,Class B,Issued,10/14/2021,"801 1 2 East Harding Drive\r\nUrbana, IL\r\n(4...","HARPER, CRAIG & JAMES E\r\n1173 COUNTY ROAD 24..."
4,1003 1/2 East Harding Drive,9.321212e+11,7/8/2013,Class B,Issued,10/14/2020,"1003 1 2 East Harding Drive\r\nUrbana, IL\r\n(...","WAMPLER, JOSEPH\r\nCOLONY PROPERTY MANAGEMENT\..."
...,...,...,...,...,...,...,...,...
1725,3026 East Stillwater Landing Unit 101,9.321224e+11,12/18/2017,Class B,Issued,10/14/2021,"3026 East Stillwater Landing\r\nUrbana, IL\r\n...","MINCONE, SANDY K\r\n22210 TAHOE CT\r\nSANTA CL..."
1726,1108 South Busey Avenue,9.321173e+11,12/16/2019,Class B,Issued,10/14/2021,"1108 South Busey Avenue\r\nUrbana, IL\r\n(40.1...","DOYLE, KIP & SHERI\r\n906 W DANIEL ST\r\nCHAMP..."
1727,806 Harvey Street,9.121074e+11,11/4/2011,Class B,Issued,10/14/2021,"806 Harvey Street\r\nUrbana, IL\r\n(-88.2215, ...","JONES PROPERTY MANAGEMENT, \r\n2516 PINEHURST ..."
1728,1302 East Michigan Avenue,9.221164e+11,4/18/2016,Class B,Issued,None,"1302 East Michigan Avenue\r\nUrbana, IL\r\n(-8...","DILLMAN, RONALD L\r\n906 E MICHIGAN AVE\r\nURB..."


The results are the same for both chunks, but in thie chunk above I make more effort to remove duplicated columns and rename the column in the `owner_addresses` data.

**SN: The `DISTINCT` keyword is a SQL function that works much like base R's `unique()` function. It may also come in handy when wanting to find the number of unique individuals of a particular subset.**


***


## <a name="validating"></a>Validating data

Validation means the checking of something for its accuracy. Validating data means checking a dataset for invalid or inaccurate entries. This kind of checking is important because as data workers we do not want analyses to start with bad or incorrect data. The first step consists of defining errors. Data errors or data glitches are those data entries that should not be there and may be caused by human error or machine error. Maybe not always, but often these errors are fixable with enough context and background information about the data.

We will utilize data validation strategies that could help us identify data errors in SQL syntax working with a new version of City of Urbana's [Rental Inspection Grades Listing Data as comma-separated .csv - GHE](https://github-dev.cs.illinois.edu/stat440-fa21/stat440-fa21-course-content/raw/master/data/rental-inspections-grades-data01.csv) or [Rental Inspection Grades Listing Data as comma-separated .csv - Box](https://uofi.box.com/shared/static/l9o50efbnemdnaxury4hg45cj8b2truu.csv).

### <a name="stra1"></a>Strategy 1 - Filtering and arranging

We can validate that there is a small number of properties earning an inspection grade of A since year 2019. 

In [10]:
rentals = pd.read_csv('https://uofi.box.com/shared/static/l9o50efbnemdnaxury4hg45cj8b2truu.csv')
rentals['Inspection Date'] = pd.to_datetime(rentals['Inspection Date']).astype(str)
rentals['Expiration Date'] = pd.to_datetime(rentals['Expiration Date']).astype(str)

import numpy as np
# this part is the same as the last part in week08 notes
n = len(rentals['Mappable Address'].iloc[0].split('\n'))
# use condition to justify whether it is nan value
addr = rentals['Mappable Address'].apply(lambda x: {i: x.split('\n')[i] 
                                                            if i<len(x.split('\n')) else np.nan for i in range(n) })
Coordinates = pd.DataFrame()
for i in range(n):
    Coordinates[str(i)] = [x[i] for x in addr]
Coordinates.iloc[:,0] = Coordinates.iloc[:,0].str.replace('1\s2','1/2')
Coordinates['City'] = 'Urbana'
Coordinates['State'] = 'IL'
Coordinates = Coordinates.drop(columns=['1'])

Coordinates00 = Coordinates['2'].str.replace('\)|\,|\(','')
Coordinates000 = Coordinates00.str.split(" ")
rentals['Coordinates01'] = [float(x[0]) if x==x else np.nan for x in Coordinates000]
rentals['Coordinates02'] = [float(x[1]) if x==x else np.nan for x in Coordinates000]
c01 = [float(x[1]) if ((x==x) and (float(x[0])<0)) else x[0] if x==x else np.nan for x in Coordinates000]
c02 = [float(x[0]) if ((x==x) and (float(x[1])>0)) else x[1] if x==x else np.nan for x in Coordinates000]
rentals2 = rentals
rentals2['Latitude'] = c01
rentals2['Longitude'] = c02
rentals2['City'] = 'Urbana'
rentals2['State'] = 'IL'
rentals2 = rentals2.drop(columns=['Mappable Address','Coordinates01','Coordinates02'])

rentals = rentals2

rentals.to_sql('rentals', conn, if_exists='replace', index = False)
pd.read_sql_query("SELECT *\
      FROM rentals\
      WHERE Grade=='Class A' AND `Inspection Date`>'2018-12-31'\
      ORDER BY `Inspection Date`;",conn)

C:\Users\kinson2\AppData\Local\Temp/ipykernel_9996/1904069054.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  Coordinates.iloc[:,0] = Coordinates.iloc[:,0].str.replace('1\s2','1/2')
C:\Users\kinson2\AppData\Local\Temp/ipykernel_9996/1904069054.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  Coordinates00 = Coordinates['2'].str.replace('\)|\,|\(','')
C:\Users\kinson2\Anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Latitude,Longitude,City,State
0,502 West Green Street Unit 303,922117127038,2019-02-28,Class A,Issued,2021-10-14,40.1108360291,-88.2136077881,Urbana,IL
1,402 West Washington Street,922117181015,2019-03-08,Class A,Expired,2020-10-14,40.105774,-88.21185,Urbana,IL
2,402 South Busey Avenue,922117108001,2019-03-20,Class A,Issued,2021-10-14,40.109623,-88.21739,Urbana,IL
3,502 West Nevada Street,922117161025,2019-04-03,Class A,Expired,2020-10-14,40.10675,-88.2133,Urbana,IL
4,412 West Washington Street,922117181004,2019-04-04,Class A,Issued,2021-10-14,40.105758667,-88.212928772,Urbana,IL
5,511 West California Avenue,922117177001,2019-04-30,Class A,Issued,2021-10-14,40.1080627441,-88.2143249512,Urbana,IL
6,203 West Nevada Street,922117260003,2019-05-15,Class A,Issued,2021-10-14,40.106346,-88.209206,Urbana,IL
7,506 West Iowa Street,932117326017,2019-05-31,Class A,Issued,2021-10-14,40.1049079895,-88.2134933472,Urbana,IL
8,304 West Elm Street,922117135013,2019-06-27,Class A,Issued,2021-10-14,40.111782074,-88.2107467651,Urbana,IL
9,805 South Race Street,922117260005,2019-07-11,Class A,Issued,2021-10-14,40.10621,-88.20899,Urbana,IL


### <a name="stra2"></a>Strategy 2 - Counting frequencies and duplicates

With the `rentals` data, the Parcel Number column is the ID variable so we can figure out duplicates easily.


In [11]:
pd.read_sql_query("SELECT COUNT(`Parcel Number`) AS cpn \
FROM rentals \
GROUP BY `Parcel Number`\
HAVING cpn>1;",conn)

,cpn


### <a name="stra3"></a>Strategy 3 - Computing summary statistics

Let's get the minimum, median, maximum, mean, standard deviation, and frequency of NAs for the coordinates - latitude and longitude.

In [12]:
#Latitude only
#minimum
lat01 = pd.read_sql_query("SELECT MIN(Latitude) AS stat, \
        CASE \
          WHEN MIN(Latitude) THEN 'minimum'\
        END AS Latitude_stat\
      FROM rentals;",conn)
lat01.to_sql('lat01', conn, if_exists='replace', index = False)

#median
m = pd.read_sql_query("SELECT Latitude \
            FROM rentals \
            WHERE Latitude IS NOT NULL \
            ORDER BY Latitude \
            LIMIT 865;", conn)
m.to_sql('m', conn, if_exists='replace', index = False)
lat02 = pd.read_sql_query("SELECT Latitude AS stat, \
        CASE \
          WHEN Latitude THEN 'median'\
        END AS Latitude_stat\
      FROM m\
      ORDER BY Latitude DESC\
      LIMIT 1;", conn)
lat02.to_sql('lat02', conn, if_exists='replace', index = False)
#maximum
lat03 = pd.read_sql_query("SELECT MAX(Latitude) AS stat, \
        CASE\
            WHEN MAX(Latitude) THEN 'maximum' \
        END AS Latitude_stat \
      FROM rentals;",conn)
lat03.to_sql('lat03', conn, if_exists='replace', index = False)
#mean
lat04 = pd.read_sql_query("SELECT AVG(Latitude) AS stat,\
                          CASE \
                              WHEN AVG(Latitude) THEN 'mean'\
                          END AS Latitude_stat\
                          FROM rentals;",conn)
lat04.to_sql('lat04', conn, if_exists='replace', index = False)
#standard deviation
lat05 = pd.read_sql_query("SELECT AVG((r.Latitude - l4.stat) * (r.Latitude - l4.stat)) AS stat,\
                          CASE \
                              WHEN stat THEN 'sample_variance' \
                          END AS Latitude_stat \
                          FROM rentals r, lat04 l4;", conn)
lat05.to_sql('lat05', conn, if_exists='replace', index = False)
#frequency of NAs
lat06 = pd.read_sql_query("SELECT COUNT(`Parcel Number`) AS frequencyOfNAs, \
        CASE \
          WHEN COUNT(`Parcel Number`) THEN 'frequencyOfNAs' \
        END AS Latitude_stat \
      FROM rentals \
      WHERE Latitude IS NULL;", conn)
lat06.to_sql('lat06', conn, if_exists='replace', index = False)
pd.read_sql_query("SELECT * FROM lat01 \
      UNION \
      SELECT * FROM lat02 \
      UNION \
      SELECT * FROM lat03 \
      UNION \
      SELECT * FROM lat04 \
      UNION \
      SELECT * FROM lat05 \
      UNION \
      SELECT * FROM lat06;", conn)

,stat,Latitude_stat
0,0.000087,sample_variance
1,1,frequencyOfNAs
2,40.106786,mean
3,40.07518,minimum
4,40.1075,median
5,40.1305,maximum


In [13]:
#Longitude only
#minimum
lon01 = pd.read_sql_query("SELECT MIN(Longitude) AS stat, \
        CASE \
          WHEN MIN(Longitude) THEN 'minimum'\
        END AS Longitude_stat\
      FROM rentals;",conn)
lon01.to_sql('lon01', conn, if_exists='replace', index = False)

#median
m = pd.read_sql_query("SELECT Longitude \
            FROM rentals \
            WHERE Longitude IS NOT NULL \
            ORDER BY Longitude \
            LIMIT 865;", conn)
m.to_sql('m', conn, if_exists='replace', index = False)
lon02 = pd.read_sql_query("SELECT Longitude AS stat, \
        CASE \
          WHEN Longitude THEN 'median'\
        END AS Longitude_stat\
      FROM m\
      ORDER BY Longitude DESC\
      LIMIT 1;", conn)
lon02.to_sql('lon02', conn, if_exists='replace', index = False)
#maximum
lon03 = pd.read_sql_query("SELECT MAX(Longitude) AS stat, \
        CASE\
            WHEN MAX(Longitude) THEN 'maximum' \
        END AS Longitude_stat \
      FROM rentals;",conn)
lon03.to_sql('lon03', conn, if_exists='replace', index = False)
#mean
lon04 = pd.read_sql_query("SELECT AVG(Longitude) AS stat, \
                          CASE \
                              WHEN AVG(Longitude) THEN 'mean'\
                          END AS Longitude_stat\
                          FROM rentals;",conn)
lon04.to_sql('lon04', conn, if_exists='replace', index = False)
#standard deviation
lon05 = pd.read_sql_query("SELECT AVG((r.Longitude - l4.stat) * (r.Longitude - l4.stat)) AS stat, \
                          CASE \
                              WHEN stat THEN 'sample_variance' \
                          END AS Longitude_stat \
                          FROM rentals r, lon04 l4;", conn)
lon05.to_sql('lon05', conn, if_exists='replace', index = False)
#frequency of NAs
lon06 = pd.read_sql_query("SELECT COUNT(`Parcel Number`) AS frequencyOfNAs, \
        CASE \
          WHEN COUNT(`Parcel Number`) THEN 'frequencyOfNAs' \
        END AS Longitude_stat \
      FROM rentals \
      WHERE Longitude IS NULL;", conn)
lon06.to_sql('lon06', conn, if_exists='replace', index = False)
pd.read_sql_query("SELECT * FROM lon01 \
      UNION \
      SELECT * FROM lon02 \
      UNION \
      SELECT * FROM lon03 \
      UNION \
      SELECT * FROM lon04 \
      UNION \
      SELECT * FROM lon05 \
      UNION \
      SELECT * FROM lon06;", conn)

,stat,Longitude_stat
0,-88.202866,mean
1,0.000193,sample_variance
2,1,frequencyOfNAs
3,-88.1633,minimum
4,-88.2033,median
5,-88.2289,maximum


***


## <a name="cleaning"></a>Cleaning data

You need to know context and background information about the data to truly fix data errors. Guessing is not appropriate, especially when money or lives are at stake. Data cleaning is a data-specific task that can be tedious and painful. Do expect to spend a long time (your allotted time multiplied by 2) on data validating and cleaning. Careful and methodical fixing of errors may yield wondrous results for your analytics team (but don't spend too much time!).


### <a name="appr1"></a>Approach 1 - Removing duplicate observations

Removing duplicate observations can be accomplished via **filtering** after successfully identifying the unique ID variable. For example, if we were to remove duplicate properties based on a newly created unique ID variable. We could combine the already unique ID variable Parcel Number with Inspection Date or we could just use Parcel Number like below.


In [14]:
pd.read_sql_query("SELECT COUNT(`Parcel Number`) AS count_id\
      FROM rentals\
      GROUP BY `Parcel Number`\
      HAVING count_id >1;",conn)

,count_id


### <a name="appr2"></a>Approach 2 - Fixing rounding errors and inconsistent units of measurement

We could round the latitude and longitude values of `rentals` data to the hundredths place via rounding with the `ROUND()` keyword.

In [15]:
pd.read_sql_query("SELECT ROUND(Latitude, 2) AS lat1, ROUND(Longitude, 2) AS lon1\
      FROM rentals;", conn)

,lat1,lon1
0,40.11,-88.19
1,40.12,-88.23
2,NaN,NaN
3,40.09,-88.20
4,40.09,-88.20
...,...,...
1725,40.09,-88.17
1726,40.10,-88.22
1727,40.12,-88.22
1728,40.10,-88.19


### <a name="appr3"></a>Approach 3 - Removing or replacing missing values

SQL has the `IS NOT NULL` keyword which quickly removes NAs from a dataset when used in the `WHERE` clause.

We could apply the `IS NOT NULL` for all columns. *Again, this is not something that is necessary for this particular dataset.**

In [16]:
pd.read_sql_query("SELECT `Property Address`, `Parcel Number`, `Inspection Date`, Grade, `License Status`, \
`Expiration Date`, Latitude, Longitude, City, State\
      FROM rentals \
      WHERE `Property Address` IS NOT NULL AND `Parcel Number` IS NOT NULL AND `Inspection Date` IS NOT NULL \
      AND Grade IS NOT NULL AND `License Status` IS NOT NULL AND `Expiration Date` IS NOT NULL AND \
                  Latitude IS NOT NULL AND Longitude IS NOT NULL AND City IS NOT NULL AND State IS NOT NULL\
      ;",conn)

,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Latitude,Longitude,City,State
0,607 1/2 Glover Avenue,922116177018,2015-07-24,Class B,Expired,2021-10-14,40.108023,-88.193322,Urbana,IL
1,1302 1/2 Hill Street,912107406011,2011-08-17,Class B,Issued,2021-10-14,40.119327,-88.226119,Urbana,IL
2,801 1/2 East Harding Drive,932121153003,2013-06-12,Class B,Issued,2021-10-14,40.093806,-88.19767,Urbana,IL
3,1003 1/2 East Harding Drive,932121153010,2013-07-08,Class B,Issued,2020-10-14,40.093743,-88.195595,Urbana,IL
4,1204 1/2 North Goodwin Avenue,912107276001,2011-10-20,Class B,Issued,2021-10-14,40.123423,-88.22415,Urbana,IL
...,...,...,...,...,...,...,...,...,...,...
1724,3026 East Stillwater Landing Unit 101,932122406009,2017-12-18,Class B,Issued,2021-10-14,40.090282,-88.16565,Urbana,IL
1725,1108 South Busey Avenue,932117307003,2019-12-16,Class B,Issued,2021-10-14,40.1031074524,-88.2173538208,Urbana,IL
1726,806 Harvey Street,912107428001,2011-11-04,Class B,Issued,2021-10-14,40.1198,-88.2215,Urbana,IL
1727,1302 East Michigan Avenue,922116376032,2016-04-18,Class B,Issued,NaT,40.1017,-88.1929,Urbana,IL


### <a name="appr4"></a>Approach 4 - Limiting a distribution to its realistic set of observations

For example, if we were to remove incorrect coordinate values. **This is not something we should do for this particular dataset.**

In [17]:
pd.read_sql_query("SELECT * \
      FROM rentals \
      WHERE 30 < Latitude < 50 OR -90 < Longitude < -70 \
      ;",conn)

,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Latitude,Longitude,City,State
0,607 1/2 Glover Avenue,922116177018,2015-07-24,Class B,Expired,2021-10-14,40.108023,-88.193322,Urbana,IL
1,1302 1/2 Hill Street,912107406011,2011-08-17,Class B,Issued,2021-10-14,40.119327,-88.226119,Urbana,IL
2,801 1/2 East Harding Drive,932121153003,2013-06-12,Class B,Issued,2021-10-14,40.093806,-88.19767,Urbana,IL
3,1003 1/2 East Harding Drive,932121153010,2013-07-08,Class B,Issued,2020-10-14,40.093743,-88.195595,Urbana,IL
4,1204 1/2 North Goodwin Avenue,912107276001,2011-10-20,Class B,Issued,2021-10-14,40.123423,-88.22415,Urbana,IL
...,...,...,...,...,...,...,...,...,...,...
1724,3026 East Stillwater Landing Unit 101,932122406009,2017-12-18,Class B,Issued,2021-10-14,40.090282,-88.16565,Urbana,IL
1725,1108 South Busey Avenue,932117307003,2019-12-16,Class B,Issued,2021-10-14,40.1031074524,-88.2173538208,Urbana,IL
1726,806 Harvey Street,912107428001,2011-11-04,Class B,Issued,2021-10-14,40.1198,-88.2215,Urbana,IL
1727,1302 East Michigan Avenue,922116376032,2016-04-18,Class B,Issued,NaT,40.1017,-88.1929,Urbana,IL


### <a name="appr5"></a>Approach 5 - Correcting and subsetting with dates

This might also be considered subsetting and applying functions and operators for dates and times.

In [18]:
pd.read_sql_query("SELECT * \
        FROM rentals \
        WHERE `Inspection Date` > '2019-12-31' AND `Inspection Date`< '2021-01-01' \
        ;", conn)

,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Latitude,Longitude,City,State
0,308 East Iowa Street,932117426024,2020-01-17,Class B,Issued,2021-10-14,40.1049842834,-88.2038574219,Urbana,IL
1,308 East Oregon Street,922117280010,2020-03-04,Class B,Issued,2021-10-14,40.1078109741,-88.2038803101,Urbana,IL
2,605 West Iowa Street,932117306016,2020-02-24,Class B,Issued,2021-10-14,40.104366,-88.21508,Urbana,IL
3,505 South Maple Street,922117277009,2020-01-07,Class B,Issued,2021-10-14,40.1087875366,-88.2027435303,Urbana,IL
4,205 West Pennsylvania Avenue,932117381007,2020-02-25,Class B,Issued,2021-10-14,40.100433,-88.21027,Urbana,IL
5,712 South Maple Street,922117286009,2020-02-12,Class B,Issued,2021-10-14,40.1066246033,-88.2021255493,Urbana,IL
6,1506 South Vine Street,932117478009,2020-03-03,Class B,Issued,2021-10-14,40.0990257263,-88.204536438,Urbana,IL
7,710 South Maple Street,922117286008,2020-01-14,Class C,Issued,2021-10-14,40.1067695618,-88.2021255493,Urbana,IL
8,206 West Florida Avenue,932117383017,2020-02-27,Class B,Issued,2021-10-14,40.0985336304,-88.2102432251,Urbana,IL
9,800 South Vine Street,922117284041,2020-01-29,Class B,Issued,2021-10-14,40.106474873,-88.204573905,Urbana,IL


### <a name="appr6"></a>Approach 6 - Correcting misspelled words, abbreviations, or text cases

We can place all property addresses, grade, and cities in all caps.

In [19]:
pd.read_sql_query("SELECT *, UPPER(`Property Address`), UPPER(`Grade`), UPPER(`City`)\
      FROM rentals\
      ;", conn)

,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Latitude,Longitude,City,State,UPPER(`Property Address`),UPPER(`Grade`),UPPER(`City`)
0,607 1/2 Glover Avenue,922116177018,2015-07-24,Class B,Expired,2021-10-14,40.108023,-88.193322,Urbana,IL,607 1/2 GLOVER AVENUE,CLASS B,URBANA
1,1302 1/2 Hill Street,912107406011,2011-08-17,Class B,Issued,2021-10-14,40.119327,-88.226119,Urbana,IL,1302 1/2 HILL STREET,CLASS B,URBANA
2,212 1/2 Central Avenue,912108383001,2010-04-26,Class B,Issued,NaT,None,None,Urbana,IL,212 1/2 CENTRAL AVENUE,CLASS B,URBANA
3,801 1/2 East Harding Drive,932121153003,2013-06-12,Class B,Issued,2021-10-14,40.093806,-88.19767,Urbana,IL,801 1/2 EAST HARDING DRIVE,CLASS B,URBANA
4,1003 1/2 East Harding Drive,932121153010,2013-07-08,Class B,Issued,2020-10-14,40.093743,-88.195595,Urbana,IL,1003 1/2 EAST HARDING DRIVE,CLASS B,URBANA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1725,3026 East Stillwater Landing Unit 101,932122406009,2017-12-18,Class B,Issued,2021-10-14,40.090282,-88.16565,Urbana,IL,3026 EAST STILLWATER LANDING UNIT 101,CLASS B,URBANA
1726,1108 South Busey Avenue,932117307003,2019-12-16,Class B,Issued,2021-10-14,40.1031074524,-88.2173538208,Urbana,IL,1108 SOUTH BUSEY AVENUE,CLASS B,URBANA
1727,806 Harvey Street,912107428001,2011-11-04,Class B,Issued,2021-10-14,40.1198,-88.2215,Urbana,IL,806 HARVEY STREET,CLASS B,URBANA
1728,1302 East Michigan Avenue,922116376032,2016-04-18,Class B,Issued,NaT,40.1017,-88.1929,Urbana,IL,1302 EAST MICHIGAN AVENUE,CLASS B,URBANA


In [20]:
conn.close()

## <a name="subqueries"></a>SQL Subqueries

Let's import the [Rental Inspection Grades Listing Data as comma-separated .csv - GHE](https://github-dev.cs.illinois.edu/stat440-fa21/stat440-fa21-course-content/raw/master/data/rental-inspections-grades-data01.csv) or [Rental Inspection Grades Listing Data as comma-separated .csv - Box](https://uofi.box.com/shared/static/l9o50efbnemdnaxury4hg45cj8b2truu.csv).


In [21]:
import pandas as pd
rentals = pd.read_csv('https://uofi.box.com/shared/static/l9o50efbnemdnaxury4hg45cj8b2truu.csv')
rentals['Inspection Date'] = pd.to_datetime(rentals['Inspection Date']).astype(str)
rentals['Expiration Date'] = pd.to_datetime(rentals['Expiration Date']).astype(str)
rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Property Address  1730 non-null   object
 1   Parcel Number     1730 non-null   int64 
 2   Inspection Date   1730 non-null   object
 3   Grade             1730 non-null   object
 4   License Status    1730 non-null   object
 5   Expiration Date   1730 non-null   object
 6   Mappable Address  1730 non-null   object
dtypes: int64(1), object(6)
memory usage: 94.7+ KB


In [22]:
import sqlite3
conn = sqlite3.connect("stat440_fa21_notes_week12_2.db") 
conn.commit()
rentals.to_sql('rentals', conn, if_exists='replace', index = False)

C:\Users\kinson2\Anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


An SQL subquery is simply a query within a query. The subquery can be written anywhere so long as the SQL syntax is followed. Here's a reminder of the proper SQL syntax for any query.

```
SELECT object-item <, ...object-item> 
FROM from-list 
<WHERE sql-expression> 
<GROUP BY object-item <, ...object-item >> 
<HAVING sql-expression> 
<ORDER BY order-by-item <DESC> <, ...order-by-item>>
```

The `SELECT` statement specifies which columns need to be in the resulting table once the query is complete. New variables may be created in the `SELECT` statement. 

The `FROM` clause points the dataset (the source of the query). Usually, this dataset exists within the database.

The `WHERE` clause is a way to select records based on conditions.

The `GROUP BY` clause groups data for processing. It helps to remove duplicates when there is a unique identifier in the data. 

The `HAVING` clause allows conditions to be placed on the groups for group processing. This clause is relevant when a `GROUP BY` clause exists.

The `ORDER BY` clause is the way to arrange the data. The default behavior is sorting in ascending order.

The order of these statements, clauses, and keywords matters and must be strictly followed. Meaning, a `WHERE` clause cannot appear before a `FROM` clause. Almost every query requires a `SELECT` statement.

## <a name="example01"></a>Example 1

One usage that I like is to put a subquery in the place of a dataset name in the `FROM` clause. For example, without a subquery, I could write two queries to show the number of unique parcel numbers of the rentals data in which the inspection grades are only A and the inspection date is after year 2018. See this chunk below.


In [23]:
x = pd.read_sql_query("SELECT * \
       FROM rentals \
       WHERE Grade=='Class A' AND `Inspection Date` > '2018-12-31' \
       ORDER BY `Inspection Date`;", conn)
x.to_sql('x', conn, if_exists='replace', index = False)

In [24]:
pd.read_sql_query("SELECT COUNT(`Parcel Number`) AS pn_count \
      FROM x \
      LIMIT 10;", conn)

,pn_count
0,20


With a subquery, this can be done as seen in this chunk below.

In [25]:
pd.read_sql_query("SELECT COUNT(`Parcel Number`) AS pn_count \
      FROM (SELECT * \
      FROM rentals \
      WHERE Grade=='Class A' AND `Inspection Date`>'2018-12-31' \
      ORDER BY `Inspection Date`) \
      LIMIT 10;", conn)

,pn_count
0,20


## <a name="example02"></a>Example 2

Another common usage is to place a subquery beginning in the `WHERE` clause. This usage might make sense for working with different datasets and filtering certain datasets to join them with another dataset. See the chunk below in which we are joining the column of a filtered owners addresses with rentals data. Particularly, we can filter the owners addresses data to keep the observations that contain the string 'THE'.

Recall the [owners-addresses .csv - GHE](https://github-dev.cs.illinois.edu/stat440-fa21/stat440-fa21-course-content/raw/master/data/owners-addresses.csv) or the Box data URL https://uofi.box.com/shared/static/u6coxibtzx3mith23bzk4rysu923g160.csv. We must first mutate to add the parcel numbers column to the existing column of the owners addresses data.


In [26]:
owners_addresses = pd.read_csv('https://uofi.box.com/shared/static/u6coxibtzx3mith23bzk4rysu923g160.csv')
owners_addresses.loc[:,'Parcel Number'] = rentals['Parcel Number']
owners_addresses.to_sql('owners_addresses', conn, if_exists='replace', index = False)

C:\Users\kinson2\Anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [27]:
pd.read_sql_query("SELECT * \
      FROM rentals \
      WHERE `Parcel Number` IN \
      (SELECT `Parcel Number` \
      FROM owners_addresses \
      WHERE value LIKE 'THE%');", conn)

,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Mappable Address
0,701 North Busey Avenue,912108302024,2014-02-06,Class B,Issued,2021-10-14,"701 North Busey Avenue\r\nUrbana, IL\r\n(-88.2..."
1,705 North Coler Avenue,912108305016,2014-02-12,Class B,Issued,2021-10-14,"705 North Coler Avenue\r\nUrbana, IL\r\n(-88.2..."
2,810 North Coler Avenue,912108308001,2014-03-19,Class B,Issued,2021-10-14,"810 North Coler Avenue\r\nUrbana, IL\r\n(-88.2..."
3,1503 East Florida Avenue,932121201013,2017-11-30,Class B,Issued,2021-10-14,"1503 East Florida Avenue\r\nUrbana, IL\r\n(40...."
4,704 North Coler Avenue,912108308009,2014-04-09,Class B,Issued,2021-10-14,"704 North Coler Avenue\r\nUrbana, IL\r\n(-88.2..."
5,706 North Coler Avenue,912108308008,2010-08-03,Class B,Issued,2021-10-14,"706 North Coler Avenue\r\nUrbana, IL\r\n(-88.2..."
6,707 North Orchard Street,912108308017,2014-04-09,Class B,Issued,NaT,"707 North Orchard Street\r\nUrbana, IL\r\n(-88..."
7,808 West University Avenue,912108304013,2014-04-09,Class B,Issued,2021-10-14,"808 West University Avenue\r\nUrbana, IL\r\n(-..."
8,704 North Lincoln Avenue,912108302004,2014-01-23,Class B,Issued,2021-10-14,"704 North Lincoln Avenue\r\nUrbana, IL\r\n(-88..."
9,708 North Busey Avenue,912108305008,2014-02-12,Class B,Temporarily Not a Rental,2021-10-14,"708 North Busey Avenue\r\nUrbana, IL\r\n(-88.2..."


## <a name="example03"></a>Example 3

A subquery beginning in the `WHERE` clause might also be useful filtering a single dataset such as the code chunk below.


In [28]:
pd.read_sql_query("SELECT * \
      FROM rentals \
      WHERE `Parcel Number` NOT IN \
    (SELECT `Parcel Number` \
     FROM rentals \
     WHERE Grade = 'Class A') \
     LIMIT 10;", conn)

,Property Address,Parcel Number,Inspection Date,Grade,License Status,Expiration Date,Mappable Address
0,607 1/2 Glover Avenue,922116177018,2015-07-24,Class B,Expired,2021-10-14,"607 1 2 Glover Avenue\r\nUrbana, IL\r\n(40.108..."
1,1302 1/2 Hill Street,912107406011,2011-08-17,Class B,Issued,2021-10-14,"1302 1 2 Hill Street\r\nUrbana, IL\r\n(40.1193..."
2,212 1/2 Central Avenue,912108383001,2010-04-26,Class B,Issued,NaT,"212 1 2 Central Avenue\r\nUrbana, IL"
3,801 1/2 East Harding Drive,932121153003,2013-06-12,Class B,Issued,2021-10-14,"801 1 2 East Harding Drive\r\nUrbana, IL\r\n(4..."
4,1003 1/2 East Harding Drive,932121153010,2013-07-08,Class B,Issued,2020-10-14,"1003 1 2 East Harding Drive\r\nUrbana, IL\r\n(..."
5,1204 1/2 North Goodwin Avenue,912107276001,2011-10-20,Class B,Issued,2021-10-14,"1204 1 2 North Goodwin Avenue\r\nUrbana, IL\r\..."
6,910 1/2 North Busey Avenue,912108153017,2010-12-17,Class B,Issued,2021-10-14,"910 1 2 North Busey Avenue\r\nUrbana, IL\r\n(4..."
7,1109 1/2 East Main Street,922116126007,2015-06-05,Class B,Issued,2020-10-14,"1109 1 2 East Main Street\r\nUrbana, IL\r\n(40..."
8,1306 1/2 East Mumford Drive,932121327003,2013-07-08,Class B,Issued,2021-10-14,"1306 1 2 East Mumford Drive\r\nUrbana, IL\r\n(..."
9,709 1/2 South Vine Street,922117264013,2009-12-07,Class D,Issued,2021-10-14,"709 1 2 South Vine Street\r\nUrbana, IL\r\n(40..."


## <a name="example04"></a>Example 4

Here's an example of using a subquery beginning in the `SELECT` clause to mutate a new column. In the code chunk below, we are mutating the rentals data with a new column containing the string "Champaign" representing the county in which all properties belong.


In [29]:
pd.read_sql_query("SELECT `Parcel Number`, `Property Address`, \
      (SELECT 'Champaign' \
      FROM rentals) AS County \
      FROM rentals \
      LIMIT 10;", conn)

,Parcel Number,Property Address,County
0,922116177018,607 1/2 Glover Avenue,Champaign
1,912107406011,1302 1/2 Hill Street,Champaign
2,912108383001,212 1/2 Central Avenue,Champaign
3,932121153003,801 1/2 East Harding Drive,Champaign
4,932121153010,1003 1/2 East Harding Drive,Champaign
5,912107276001,1204 1/2 North Goodwin Avenue,Champaign
6,912108153017,910 1/2 North Busey Avenue,Champaign
7,922116126007,1109 1/2 East Main Street,Champaign
8,932121327003,1306 1/2 East Mumford Drive,Champaign
9,912108354004,807 1/2 West Main Street,Champaign


In [30]:
conn.close()

#### END OF NOTES